In [1]:
import os
import tifffile
import numpy as np
import scipy.io as sio
import warnings
import scipy.ndimage as ndi
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import sys
import re
from skimage.filters import gaussian, median
from skimage import io
from skimage.segmentation import watershed
from skimage.feature import hessian_matrix
from skimage.measure import *
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from skimage.morphology import *
from matplotlib.widgets import Button, TextBox
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict, Any, Union
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

In [5]:
size_lim = [256, 256, 149]
clusters = 6 # Numero di cluster per il k-means (default = 6)
path_img = 'test_data.tif'
sig_G = [0]

# Raccogli informazioni dall'intestazione dell'immagine
with tifffile.TiffFile('test_data.tif') as tif:
    # Get the image array
    info1 = tif.asarray()
Nz = len(info1)
Ny = info1[0].shape[1]
Nx = info1[0].shape[0]
tp = info1[0].dtype

if tp == 'uint16': 
    bit = 16
elif tp == 'uint8':
    bit = 8
else:
    raise ValueError('input image must be 8 or 16 bit')

nxiL = list(range(0, Nx, size_lim[0]))
nxeL = [min(Nx, i) for i in range(size_lim[0], Nx + size_lim[0], size_lim[0])]
nyiL = list(range(0, Ny, size_lim[1]))
nyeL = [min(Ny, i) for i in range(size_lim[1], Ny + size_lim[1], size_lim[1])]
th_back = 0.02 * (2 ** bit)
win = size_lim[2]  # Dimensione del ritaglio lungo l'asse z
safe = 3  # Margine z
safe_xy = 3  # Margine xy
k_seq = list(range(0, Nz, win))  # Numero di fette dell'immagine iniziale lungo l'asse z

In [107]:
tags = tif.pages[0].tags


In [108]:
res_tags = ['XResolution', 'YResolution']
res = [1, 1, 1]
for i,res_tag in enumerate(res_tags):
    if res_tag in tags:
        res[i] = 1 / (tags[res_tag].value[0] / tags[res_tag].value[1])

strinfo = tags['ImageDescription'].value
um = strinfo.find('spacing=')
cut_s = len('spacing=')
res[2] = zres = float(re.search(r'\d+\.\d+', strinfo[um + cut_s:]).group(0))


In [140]:
for nxi,nxe in zip(nxiL, nxeL):
    # Definisci gli indici per il ritaglio
    nxiS = max(0, nxi - safe_xy)
    nxeS = min(Nx, nxe + safe_xy)
    NxC = nxe - nxi

    # Ciclo sull'asse y
    for nyi,nye in zip(nyiL, nyeL):
        # Definisci gli indici per il ritaglio
        nyiS = max(0, nyi - safe_xy)
        nyeS = min(Ny, nye + safe_xy)
        NyC = nye - nyi

        # Ciclo sull'asse z
        for k in k_seq:
            # Definisci gli indici per il ritaglio
            xinit = max(0, k - safe)
            xend = min(Nz, k + win + safe)
            NzC = min(Nz, k + win) - k

            # Controlla se il ritaglio è già stato elaborato, in tal caso, continua...
            
            # Visualizza messaggi di stato
            print(f'number of clusters is: {clusters}')
            print(f'crop is x({nxi}:{nxe}), y({nyi}:{nye})')
            print(f'starting with slice {k} over {Nz}, window set to {win}')

            # Leggi il ritaglio dell'immagine 
            # (i) per gestire immagini molto grandi;
            # (ii) prima definisci la dimensione effettiva del ritaglio, 
            
            nx = len(range(nxiS, nxeS))
            ny = len(range(nyiS, nyeS))
            nz = xend - xinit
            cIMc = np.zeros((nx, ny, nz), dtype=tp)

            # (iii) itera sulle fette dell'immagine originale per leggere il ritaglio

            for zz in range(cIMc.shape[2]):
                cIMc[:, :, zz] = tifffile.imread(path_img, key=xinit + zz - 1)[nxiS:nxeS, nyiS:nyeS]

            # Filtro mediano se sig_G[0] == -1
            if sig_G[0] == -1:
                print('applying 3x3x3 median filter...')
                cIMc = median(cIMc, size=3)

            # Fornisce feedback all'utente
            print(f'1st of {len(sig_G)} passes...')

            # Derivate di primo ordine
            if sig_G[0] > 0:
                cIMc_smoothed = gaussian(cIMc.astype(np.float32), sigma=[sig_G[0], sig_G[0] * res[0] / res[1], sig_G[0] * res[0] / res[2]])
                Gx2, Gy2, Gz2 = np.gradient(cIMc_smoothed)
            else:
                Gx2, Gy2, Gz2 = np.gradient(cIMc.astype(np.float32))

            # Derivate di secondo ordine
            Hxx, Hxy, Hxz, Hyy, Hyz, Hzz = hessian_matrix(Gx2, sigma=1)
            Gxx2 = Hxx
            Gyy2 = Hyy
            Gzz2 = Hzz

            cIMc = cIMc[
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), cIMc.shape[0]),
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), cIMc.shape[1]),
                min(safe_xy,k):min(win+min(safe_xy,k), cIMc.shape[2]),
            ]

            # su GxxKt
            GxxKt = Gxx2[
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), Gxx2.shape[0]),
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), Gxx2.shape[1]),
                min(safe_xy,k):min(win+min(safe_xy,k), Gxx2.shape[2]),
            ]

            # su GyyKt
            GyyKt = Gyy2[
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), Gyy2.shape[0]),
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), Gyy2.shape[1]),
                min(safe_xy,k):min(win+min(safe_xy,k), Gyy2.shape[2]),
            ]

            # su GzzKt
            GzzKt = Gzz2[
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), Gzz2.shape[0]),
                min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), Gzz2.shape[1]),
                min(safe_xy,k):min(win+min(safe_xy,k), Gzz2.shape[2]),
            ]

            # Salva il ritaglio corrente dell'immagine
            with open(os.path.join(f'sl{k}_{nxi}_{nxe}_{nyi}_{nye}.pkl'), 'wb') as f:
                pickle.dump({
                    'cIMc': cIMc,
                    'GxxKt': GxxKt,
                    'GyyKt': GyyKt,
                    'GzzKt': GzzKt
                }, f)
            
            # Maschera il background a intensità quasi zero per ridurre l'uso della memoria
            mask_back = np.where(cIMc.ravel() >= th_back)[0]
            resiz_km = np.ones(len(cIMc.ravel()), dtype=np.uint8)

            # Definisci lo spazio delle caratteristiche per il k-means
            # Ensure the feature vectors are properly scaled
            km_in1 = np.column_stack((
                cIMc.ravel()[mask_back].astype(np.float32),
                GxxKt.ravel()[mask_back],
                GyyKt.ravel()[mask_back],
                GzzKt.ravel()[mask_back]
            ))

            # Initialize TOT_KM1 with proper shape
            TOT_KM1 = np.ones((NxC, NyC, NzC), dtype=tp)

            # Clustering k-means only if enough samples
            kmeans = KMeans(n_clusters=clusters, n_init=10, max_iter=1000, random_state=42)
            labels = kmeans.fit_predict(km_in1.astype(np.float32))
            resiz_km[mask_back] = labels.astype(np.uint8)
            TOT_KM1 = resiz_km.reshape((NxC, NyC, NzC))
                

            # Save the k-means clustering results for the current crop
            with open(os.path.join(f'sl{k}_{nxi}_{nxe}_{nyi}_{nye}.pkl'), 'ab') as f:
                    pickle.dump({'TOT_KM1': TOT_KM1}, f)

            


number of clusters is: 6
crop is x(0:256), y(0:256)
starting with slice 0 over 143, window set to 149
1st of 1 passes...


/var/folders/x1/cw0t9qw9547b5jd05cm3d2vr0000gn/T/ipykernel_97569/2747371345.py:58: FutureWarning: use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current behavior
  Hxx, Hxy, Hxz, Hyy, Hyz, Hzz = hessian_matrix(Gx2, sigma=1)


number of clusters is: 6
crop is x(0:256), y(256:512)
starting with slice 0 over 143, window set to 149
1st of 1 passes...


/var/folders/x1/cw0t9qw9547b5jd05cm3d2vr0000gn/T/ipykernel_97569/2747371345.py:58: FutureWarning: use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current behavior
  Hxx, Hxy, Hxz, Hyy, Hyz, Hzz = hessian_matrix(Gx2, sigma=1)


KeyboardInterrupt: 

In [ ]:
nxi = nxiL[0]
nxe = nxeL[0]
# Definisci gli indici per il ritaglio
nxiS = max(0, nxi - safe_xy)
nxeS = min(Nx, nxe + safe_xy)
NxC = nxe - nxi

# Ciclo sull'asse y
nyi = nyiL[0]
nye = nyeL[0]
# Definisci gli indici per il ritaglio
nyiS = max(0, nyi - safe_xy)
nyeS = min(Ny, nye + safe_xy)
NyC = nye - nyi

# Ciclo sull'asse z
for k in k_seq:
    # Definisci gli indici per il ritaglio
    xinit = max(0, k - safe)
    xend = min(Nz, k + win + safe)
    NzC = min(Nz, k + win) - k

    # Controlla se il ritaglio è già stato elaborato, in tal caso, continua...
    
    # Visualizza messaggi di stato
    print(f'number of clusters is: {clusters}')
    print(f'crop is x({nxi}:{nxe}), y({nyi}:{nye})')
    print(f'starting with slice {k} over {Nz}, window set to {win}')

    # Leggi il ritaglio dell'immagine 
    # (i) per gestire immagini molto grandi;
    # (ii) prima definisci la dimensione effettiva del ritaglio, 
    
    nx = len(range(nxiS, nxeS))
    ny = len(range(nyiS, nyeS))
    nz = xend - xinit
    cIMc = np.zeros((nx, ny, nz), dtype=tp)

    # (iii) itera sulle fette dell'immagine originale per leggere il ritaglio

    for zz in range(cIMc.shape[2]):
        cIMc[:, :, zz] = tifffile.imread(path_img, key=xinit + zz - 1)[nxiS:nxeS, nyiS:nyeS]

    # Filtro mediano se sig_G[0] == -1
    if sig_G[0] == -1:
        print('applying 3x3x3 median filter...')
        cIMc = median(cIMc, size=3)

    # Fornisce feedback all'utente
    print(f'1st of {len(sig_G)} passes...')

    # Derivate di primo ordine
    if sig_G[0] > 0:
        cIMc_smoothed = gaussian(cIMc.astype(np.float32), sigma=[sig_G[0], sig_G[0] * res[0] / res[1], sig_G[0] * res[0] / res[2]])
        Gx2, Gy2, Gz2 = np.gradient(cIMc_smoothed)
    else:
        Gx2, Gy2, Gz2 = np.gradient(cIMc.astype(np.float32))

    # Derivate di secondo ordine
    Hxx, Hxy, Hxz, Hyy, Hyz, Hzz = hessian_matrix(Gx2, sigma=1)
    Gxx2 = Hxx
    Gyy2 = Hyy
    Gzz2 = Hzz

    cIMc = cIMc[
        min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), cIMc.shape[0]),
        min(safe_xy,nxi):min(NxC+min(safe_xy,nxi), cIMc.shape[1]),
        min(safe_xy,k):min(win+min(safe_xy,k), cIMc.shape[2]),
    ]
    print(f'cIMc shape: {cIMc.shape}\n')
    # su GxxKt
    GxxKt = Gxx2[
        max(0, safe_xy):min(NxC, safe_xy + NxC),
        max(0, safe_xy):min(NyC, safe_xy + NyC),
        max(0, safe):min(win, safe + win)
    ]

    # su GyyKt
    GyyKt = Gyy2[
        max(0, safe_xy):min(NxC, safe_xy + NxC),
        max(0, safe_xy):min(NyC, safe_xy + NyC),
        max(0, safe):min(win, safe + win)
    ]

    # su GzzKt
    GzzKt = Gzz2[
        max(0, safe_xy):min(NxC, safe_xy + NxC),
        max(0, safe_xy):min(NyC, safe_xy + NyC),
        max(0, safe):min(win, safe + win)
    ]

    # Salva il ritaglio corrente dell'immagine
    with open(os.path.join(f'sl{k}_{nxi}_{nxe}_{nyi}_{nye}.pkl'), 'wb') as f:
        pickle.dump({
            'cIMc': cIMc,
            'GxxKt': GxxKt,
            'GyyKt': GyyKt,
            'GzzKt': GzzKt
        }, f)
    
    # Maschera il background a intensità quasi zero per ridurre l'uso della memoria
    mask_back = np.where(cIMc.ravel() >= th_back)[0]
    resiz_km = np.ones(len(cIMc.ravel()), dtype=np.uint8)

    # Definisci lo spazio delle caratteristiche per il k-means
    # Ensure the feature vectors are properly scaled
    km_in1 = np.column_stack((
        cIMc.ravel()[mask_back].astype(np.float32),
        GxxKt.ravel()[mask_back],
        GyyKt.ravel()[mask_back],
        GzzKt.ravel()[mask_back]
    ))

    # Initialize TOT_KM1 with proper shape
    TOT_KM1 = np.ones((NxC, NyC, NzC), dtype=tp)

    # Clustering k-means only if enough samples
    if km_in1.shape[0] >= 10:
        try:
            kmeans = KMeans(n_clusters=clusters, n_init=10, max_iter=1000, random_state=42)
            labels = kmeans.fit_predict(km_in1.astype(np.float32))
            resiz_km[mask_back] = labels.astype(np.uint8)
            TOT_KM1 = resiz_km.reshape((NxC, NyC, NzC))
        except Exception as e:
            print(f"Warning: K-means clustering failed with error: {e}")

    # Save the k-means clustering results for the current crop
    try:
        with open(os.path.join(f'sl{k}_{nxi}_{nxe}_{nyi}_{nye}.pkl'), 'ab') as f:
            pickle.dump({'TOT_KM1': TOT_KM1}, f)
    except Exception as e:
        print(f"Warning: Failed to save results: {e}")


number of clusters is: 6
crop is x(0:256), y(0:256)
starting with slice 0 over 143, window set to 149
1st of 1 passes...


/var/folders/x1/cw0t9qw9547b5jd05cm3d2vr0000gn/T/ipykernel_97569/2095941355.py:60: FutureWarning: use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current behavior
  Hxx, Hxy, Hxz, Hyy, Hyz, Hzz = hessian_matrix(Gx2, sigma=1)


cIMc shape: (259, 259, 143)

cIMc shape: (256, 256, 143)



IndexError: index 8961260 is out of bounds for axis 0 with size 8961260